# Сумма цифр трёхзначного числа

(Задача из контеста по [лабе](http://cs.mipt.ru/algo/lessons/sem_2/01.recall.html).)

Дано трёхзначное число. Найдите сумму его цифр.

### Формат входных данных

Вводится трёхзначное число: не меньшее 100 и не большее 999.

### Формат выходных данных

Одно число — сумма цифр. 

### Примеры

<br/>

<div style="display: inline-block">

| Вход | Выход   |
| ---  | ------- |
| 179  | 17      |

</div>

# Решение

Возьмём число:

In [1]:
a = 2024

## 1. "Извлекание" цифр

In [2]:
d0 = a % 10         # 2024 -> 4
d1 = a % 100 // 10  # 2024 -> 24 -> 2
d2 = a // 100 % 10  # 2024 -> 20 -> 0
d3 = a // 1000      # 2024 -> 2

print('Digits:', d3, d2, d1, d0)
print('Sum:', d0 + d1 + d2 + d3)

Digits: 2 0 2 4
Sum: 8


## 2. "Отсекание" цифр

In [3]:
b = a
digits = []

while b > 0:
    d = b % 10
    digits.append(d)
    b = b // 10

print('Digits:', *digits[::-1])
print('Sum:', sum(digits))

Digits: 2 0 2 4
Sum: 8


## 3. Список символов цифр

In [4]:
char_digits = list(str(a))
digits = [int(d) for d in char_digits]

print('Digits:', *digits)
print('Sum:', sum(digits))

Digits: 2 0 2 4
Sum: 8


## 4. Список символов цифр (развёрнутый вариант)

In [5]:
str_num = str(a)
digits = []

for c in str_num:
    digits.append(int(c))

print('Digits:', *digits[::-1])
print('Sum:', sum(digits))

Digits: 4 2 0 2
Sum: 8


## 5. Re

In [6]:
import re

In [7]:
char_digits = re.findall('\d', str(a))
digits = [int(d) for d in char_digits]

print('Digits:', *digits)
print('Sum:', sum(digits))

Digits: 2 0 2 4
Sum: 8


## 6. Тупо перебор цифр, или Count

In [8]:
DIGITS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

str_num = str(a)
s = 0

for d in DIGITS:
    s = s + d * str_num.count(str(d))

print('Sum:', s)

Sum: 8


## 7. Через функцию

In [9]:
def sum_digits(n: int) -> int:
    char_digits = list(str(a))
    digits = [int(d) for d in char_digits]

    return sum(digits)


print('Sum:', sum_digits(a))

Sum: 8


## 8. Через класс

In [10]:
class Calculator:
    @staticmethod
    def sum_digits(n: int) -> int:
        char_digits = list(str(a))
        digits = [int(d) for d in char_digits]

        return sum(digits)


print('Sum:', Calculator.sum_digits(a))

Sum: 8


## 9. Eval

In [11]:
result = eval('+'.join(list(str(a))))

print('Sum:', Calculator.sum_digits(a))

Sum: 8


*P.S.*

Но вообще eval-ом пользоваться не рекомендуется, потому что это "мутно":

* https://realpython.com/python-eval-function/#minimizing-the-security-issues-of-eval
* https://stackoverflow.com/a/1832957/8094251
* https://stackoverflow.com/a/1834754/8094251

## 10*. AI

Пример основан на примерах отсюда:
* https://huggingface.co/openchat/openchat-3.6-8b-20240522
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct

In [ ]:
# Install PyTorch: https://pytorch.org/get-started/locally

! pip install transformers
! pip install accelerate

In [12]:
import os

# Turn off CUDA (we don't need it, and probably you don't have it)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [14]:
model_id = "microsoft/Phi-3.5-mini-instruct"  # ~ 7-8 Gb

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant."
    },
    {
        "role": "user",
        "content": (
            f"Дано число: {a}."
            f" Посчитай сумму его цифр."
            f" В качестве ответа скажи только одно число — значение суммы."
        )
    },
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)
input_ids = input_ids.to(model.device)

outputs = model.generate(
    input_ids,
    do_sample=False,
    max_new_tokens=10
)
response = outputs[0][input_ids.shape[-1]:]

answer = tokenizer.decode(response, skip_special_tokens=True)

print('Sum:', answer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.


Sum: 8


*P.S.*

Можно посмотреть весь вход модели как текст:

```python
tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)
```

Можно посмотреть весь выход:

```python
tokenizer.decode(outputs[0])
```

## 11*. LLLambda

In [17]:
sum_digits = lambda n: (lambda f, x: f(f, x))(lambda g, y: y % 10 + g(g, y // 10) if y > 0 else 0, n)

print('Sum:', sum_digits(a))

Sum: 8


*P.S.*

Автор ноутбука сам до такого никогда бы не додумался 😅
Решение вдохновлено воспоминаниями об:
* однострочнике по удалению дубликатов из списка: https://stackoverflow.com/a/41524134/8094251.
* и способе сделать reduce через лямбды: https://stackoverflow.com/a/44071474/8094251.